In [2]:
using Revise

In [3]:
using Plots

In [ ]:
using LaTeXStrings

In [4]:
using UnPack

In [1]:
using LinearAlgebra
using ImmersedLayers

In [5]:
"""
    HeaterConfig

A structure to hold the parameters of the heater simulations

## Fields
- `Nq::Int64`: number of heaters
- `state_id`: Look-up for state component indices
- `Δt::Float64`: time step
"""
struct HeaterConfig{SID} 
    "Number of heaters"
    Nq::Int64

    "State IDs"
    state_id::SID

    "Time step"
    Δt::Float64

end

function HeaterConfig(Nq,Δt)
  state_id = construct_heater_state_mapping(Nq)
  HeaterConfig{typeof(state_id)}(Nq,state_id,Δt)
end

function HeaterConfig(Nq)
  state_id = construct_heater_state_mapping(Nq)
  HeaterConfig{typeof(state_id)}(Nq,state_id,0.0)
end
function construct_heater_state_mapping(Nq::Int64)
    state_id = Dict()
    heater_x_ids = zeros(Int,Nq)
    heater_y_ids = zeros(Int,Nq)
    heater_q_ids = zeros(Int,Nq)
    heater_r_ids = zeros(Int,Nq)
  
    for j in 1:Nq
      heater_x_ids[j] = 4j-3
      heater_y_ids[j] = 4j-2
      heater_q_ids[j] = 4j-1
      heater_r_ids[j] = 4j
    end
  
    state_id["heater x"] = heater_x_ids
    state_id["heater y"] = heater_y_ids
    state_id["heater q"] = heater_q_ids
    state_id["heater r"] = heater_r_ids
  
    return state_id
  end

construct_heater_state_mapping (generic function with 1 method)

In [6]:
function positions_and_strengths_to_state(zq::AbstractVector{ComplexF64},qq::AbstractVector{Float64},rq::AbstractVector{Float64},config::HeaterConfig)
    @unpack Nq, state_id = config
    state = zeros(state_length(state_id))
  
    x_ids = state_id["heater x"]
    y_ids = state_id["heater y"]
    q_ids = state_id["heater q"]
    r_ids = state_id["heater r"]
  
  
    for i = 1:Nq
      state[x_ids[i]] = real(zq[i])
      state[y_ids[i]] = imag(zq[i])
      state[q_ids[i]] = qq[i]
      state[r_ids[i]] = rq[i]
    end
  
    return state
  end  

positions_and_strengths_to_state (generic function with 1 method)

In [7]:
state_length(config::HeaterConfig) =  state_length(config.state_id)

state_length(a::Dict) = mapreduce(key -> state_length(a[key]),+,keys(a))
state_length(a::Vector) = length(a)
state_length(a::Matrix) = 0
state_length(a::Int) = 0

state_length (generic function with 5 methods)

In [8]:
function get_config_and_state(zq::AbstractVector,qq::AbstractVector,rq::AbstractVector)
    Nq = length(zq)
    config = HeaterConfig(Nq)
    x = positions_and_strengths_to_state(zq,qq,rq,config)

    return config, x
end

get_config_and_state (generic function with 1 method)

In [9]:
export constructGrids, AbstractGrids

abstract type AbstractGrids end

struct constructGrids{XT,CT} <: AbstractGrids

    "Grid spacing"
    Δx::Float64

    "Size of Domain"
    Lx::Float64

    "size of domain in x-direction"
    xlim::XT

    "size of domain in y-direction"
    ylim::XT
    
    "Physical Grid"
    g::PhysicalGrid

    "Cache"
    cache::CT
end

function constructGrids(Δx,Lx)
    xlim = (-Lx/2,Lx/2)
    ylim = (-Lx/2,Lx/2)
    g = PhysicalGrid(xlim,ylim,Δx)
    cache = SurfaceScalarCache(g)
    CacheType = typeof(cache)
    constructGrids{typeof(xlim), CacheType}(Δx,Lx,xlim,ylim,g,cache)
end

constructGrids

In [10]:
abstract type AbstractObservationOperator{Nx,Ny,withsensors} end
abstract type AbstractCartesianHeaterObservations{Nx,Ny} <: AbstractObservationOperator{Nx,Ny,true} end
struct TemperatureObservations{Nx,Ny,ST,CT} <: AbstractObservationOperator{Nx,Ny,true}
	sens::ST
	config::CT
end
function TemperatureObservations(sens::AbstractVector,config::HeaterConfig)
    return TemperatureObservations{4*config.Nq,length(sens),typeof(sens),typeof(config)}(sens,config)
end

TemperatureObservations

In [11]:
function observations(x::AbstractVector,t,obs::TemperatureObservations,gridConfig::constructGrids)
    return _temperature(x,obs,gridConfig)
end
_temperature(x,obs::TemperatureObservations,gridConfig::constructGrids) = analytical_temperature(x,obs,gridConfig)
function analytical_temperature(x::AbstractVector,obs::TemperatureObservations,gridConfig::constructGrids)
	@unpack config, sens = obs
	@unpack Nq, state_id = config
	@unpack g, cache = gridConfig
	Ny = length(sens)

	x_ids = state_id["heater x"]
  	y_ids = state_id["heater y"]
  	q_ids = state_id["heater q"]
  	r_ids = state_id["heater r"]

	xq = x[x_ids]
	yq = x[y_ids]
	qq = x[q_ids]
	rq = x[r_ids]

	T = zeros_grid(cache)
	xg, yg = coordinates(T,g)
	Temp = zeros(Ny)

	for k in 1:Nq
		T = zeros_grid(cache)
		for (j,y) in enumerate(yg), (i,x) in enumerate(xg)
    		if ((x-xq[k])^2+(y-yq[k])^2) < rq[k]^2
        		T[i,j] = -qq[k]
    		end
		end
		inverse_laplacian!(T,cache)
		Tfield = interpolatable_field(T,g)
		Temp .+= [Tfield(real(sens[j]), imag(sens[j])) for j in 1:Ny]
	end
	return Temp
end

analytical_temperature (generic function with 1 method)

In [12]:
function get_truth_data(Nsens,ϵmeas,x_true::Vector,config_true,gridConfig::constructGrids;layout=(:line,1.0),add_noise=false)

    t = 0.0

    sens = setup_sensors(Nsens;layout=layout)

    # Get true observations
    obs_true = TemperatureObservations(sens,config_true)

    # evaluate data without noise
    ystar0 = observations(x_true,t,obs_true,gridConfig)


    return obs_true, ystar0, sens

end
function setup_sensors(Nsens;layout=(:line,1.0))

    layout_type, len = layout
  
    if layout_type == :circle
      rsens = len
      θsens = range(0,2π,length=Nsens+1)
      sens = rsens*exp.(im*θsens[1:end-1])
    elseif layout_type == :line
      ϵsens = 0.0
      lowerrow = range(-len,len,length=Nsens) .+ (-0.5ϵsens .+ ϵsens*rand(Nsens))*im
      #upperrow = range(-2.0,2.0,length=Nsens) .+ 1.0*im
      #leftside = im*range(-1.0,3.0,length=Nsens) .- 1.0
      #rightside = im*range(-1.0,3.0,length=Nsens) .+ 1.0
      sens = vcat(lowerrow,)  #upperrow);
    elseif layout_type == :dline
      ϵsens = 0.02
      lowerrow1 = range(-len,len,length=Nsens÷2) .- 0.5*ϵsens
      lowerrow2 = range(-len,len,length=Nsens÷2) .+ 0.5*ϵsens
      sens = sort(vcat(lowerrow1,lowerrow2)) .+ 0.0im
    end
    return sens
  end
  

setup_sensors (generic function with 1 method)

### True data

In [14]:
zq = [0.5+1.2im,-0.5+0.7im]
qq = [1.0,2.0]
rq = [0.5,0.2]
Nq_true = length(zq)

2

In [15]:
config_true, x_true = get_config_and_state(zq,qq,rq)
x_true

8-element Vector{Float64}:
  0.5
  1.2
  1.0
  0.5
 -0.5
  0.7
  2.0
  0.2

### Getting results for a specific case

In [17]:
Δx = 0.01
Lx = 4.0
gridConfig = constructGrids(Δx,Lx)

constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 404, 408, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 404, 408, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 404, 408, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 404, 408, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 404, 408, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{404, 408, Float64, true, false}, Edges{Primal, 404, 408, Float64, Vector{Float64}}, Nodes{Dual, 404, 408, Float64, Matrix{Float64}}, Nothing, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Nothing}}(0.01, 4.0, (-2.0, 2.0), (-2.

In [34]:
gridConfig.cache

Surface cache with scaling of type GridScaling
  0 point data of type ScalarData{0, Float64, Vector{Float64}}
  Grid data of type Nodes{Primal, 610, 608, Float64, Matrix{Float64}}


In [18]:
K = 5
Nsens = 3
ϵmeas = 5e-4
obs_true, ystar,sens = get_truth_data(Nsens,ϵmeas,x_true,config_true,gridConfig)

(TemperatureObservations{8, 3, Vector{ComplexF64}, HeaterConfig{Dict{Any, Any}}}(ComplexF64[-1.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im], HeaterConfig{Dict{Any, Any}}(2, Dict{Any, Any}("heater q" => [3, 7], "heater r" => [4, 8], "heater y" => [2, 6], "heater x" => [1, 5]), 0.0)), [-0.8323193110822054, -0.7836443800526118, -0.8096823306433623], ComplexF64[-1.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im])

In [16]:
sens

3-element Vector{ComplexF64}:
 -1.0 + 0.0im
  0.0 + 0.0im
  1.0 + 0.0im

In [19]:
ystar

3-element Vector{Float64}:
 -0.8323193110822054
 -0.7836443800526118
 -0.8096823306433623